## 텍스트 마이닝

### 데이터 준비

In [ ]:
# 관련 라이브러리를 호출합니다.
import os
import numpy as np
import pandas as pd
from kiwipiepy import Kiwi
from kiwipiepy.utils import Stopwords
from tqdm.notebook import tqdm
from nltk import bigrams
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
# 현재 작업 경로를 확인합니다.
os.getcwd()

In [ ]:
# data 폴더로 작업 경로를 변경합니다.
os.chdir(path = '../data')

In [ ]:
# 현재 작업 경로에 있는 폴더명과 파일명을 확인합니다.
sorted(os.listdir())

In [ ]:
# pkl 파일을 읽고 newsReply를 생성합니다.
newsReply = pd.read_pickle(filepath_or_buffer = 'Naver_News_Reply.pkl')

In [ ]:
# newsReply의 처음 5행을 확인합니다.
newsReply.head()

In [ ]:
# newsReply에서 contents의 글자수가 1 이상인 행을 남기고 행이름을 초기화합니다.
newsReply = newsReply.loc[newsReply['contents'].str.len().ge(other = 1), :].reset_index(drop = True)

In [ ]:
# newsReply에서 corrected를 선택하고 처음 10행을 출력합니다.
for i in range(10):
    print(f'{i}: {newsReply["corrected"].iloc[i]}', end = '\n\n')

In [ ]:
# 교정되지 않은 오탈자를 수작업으로 교정합니다.
newsReply['corrected'] = newsReply['corrected'].str.replace(pat = '성깔 부리고', repl = '성질부리고')
newsReply['corrected'] = newsReply['corrected'].str.replace(pat = '안 달', repl = '안달')
newsReply['corrected'] = newsReply['corrected'].str.replace(pat = '마녀 사냥', repl = '마녀사냥')
newsReply['corrected'] = newsReply['corrected'].str.replace(pat = '악위적', repl = '악의적')
newsReply['corrected'] = newsReply['corrected'].str.replace(pat = '쇄락', repl = '추락')
newsReply['corrected'] = newsReply['corrected'].str.replace(pat = '모른 행사장', repl = '모든 행사장')
newsReply['corrected'] = newsReply['corrected'].str.replace(pat = '기웃거다', repl = '기웃거리다')
newsReply['corrected'] = newsReply['corrected'].str.replace(pat = '업무 방해', repl = '업무방해')

### 형태소 분석

In [ ]:
# 한글 형태소 분석기 객체를 생성합니다.
kiwi = Kiwi(model_type = 'sbg', typos = 'basic_with_continual_and_lengthening')

In [ ]:
# 한글 맞춤법 검사 결과를 corrected에 할당합니다.
corrected = '복리후생으로 워라벨부터 챙기자.'

In [ ]:
# corrected로 형태소 분석을 실행한 결과를 확인합니다.
kiwi.tokenize(text = corrected)

### 불용어 삭제

In [ ]:
# 내장 불용어 객체를 생성합니다.
stopwords = Stopwords()

In [ ]:
# 내장 불용어 목록을 확인합니다.
stopwords.stopwords

In [ ]:
# 형태소 분석을 실행하고 불용어를 삭제한 결과를 확인합니다.
kiwi.tokenize(text = corrected, stopwords = stopwords)

### 사용자 사전 추가

In [ ]:
# 사용자 사전에 새로운 단어를 추가합니다.
kiwi.add_user_word(word = '복리후생', tag = 'NNG', score = 1)

In [ ]:
# 사용자 사전을 적용하여 형태소 분석을 실행하고 불용어를 제거합니다.
tokens = kiwi.tokenize(text = corrected, stopwords = stopwords)

In [ ]:
# tokens(형태소 분석 결과)를 확인합니다.
tokens

In [ ]:
# 첫 번째 형태소의 단어와 품사를 확인합니다.
tokens[0].form, tokens[0].tag

### 품사 선택

In [ ]:
# 형태소 분석 결과에서 선택할 품사(용언과 체언) 목록을 리스트로 생성합니다.
tag_v, tag_n = ['VV', 'VV-R', 'VV-I', 'VA', 'VA-R', 'VA-I'], ['NNG', 'NNP']

In [ ]:
# 품사가 용언과 체언인 형태소만 선택하고, 품사가 용언일 때 종결어미 '다'를 결합합니다.
[token.form + '다' if token.tag in tag_v else token.form 
 for token in tokens if token.tag in tag_v + tag_n]

### 형태소 분석 함수 생성

In [ ]:
# 문서를 입력받아 형태소 분석을 실행하고 리스트로 반환하는 함수를 생성합니다.
def tokenizer(text):
    tokens = kiwi.tokenize(text = text, stopwords = stopwords)
    return [token.form + '다' if token.tag in tag_v else token.form
            for token in tokens if token.tag in tag_v + tag_n]

### BoW 생성: 문서 집합으로 형태소 분석 실행

In [ ]:
# 형태소 분석 결과를 저장할 빈 리스트를 생성합니다.
morphs = []

# 문서 집합에서 각 문서를 형태소로 나누고 일부 품사를 남긴 morphs를 생성합니다.
for corrected in tqdm(newsReply['corrected']):
    tokens = tokenizer(text = corrected)
    morphs.append(tokens)

### BoW 확인: 사용자 사전에 추가할 단어 탐색

In [ ]:
# 한글 맞춤법 검사를 실행한 문서와 형태소 분석 결과를 비교합니다.
for i in range(10):
    print(f'{i}: {newsReply["corrected"].iloc[i]}')
    print(f'{i}: {morphs[i]}', end = '\n\n')

### 사용자 사전용 텍스트 파일 생성 및 읽기

In [ ]:
# 사용자 사전에 추가할 단어 목록과 단어의 품사 및 점수를 각각 리스트로 생성합니다.
words = ['악의적', '비위생', '대박', '업무방해', '자발적', '더본코리아']
tags = ['NNG'] * 5 + ['NNP']
scores = [1] * 6

In [ ]:
# 사용자 사전 파일명을 지정합니다.
dict_file = 'User_Dictionary.txt'

In [ ]:
# 사용자 사전을 텍스트 파일로 저장합니다.
# [참고] 단어, 품사, 점수 사이에 구분자(탭)를 설정하고 문장 마지막에 개행문자를 추가합니다.
with open(file = dict_file, mode = 'w') as file:
    for word, tag, score in zip(words, tags, scores):
        file.write(f'{word}\t{tag}\t{score}\n')

In [ ]:
# 사용자 사전에 등록할 텍스트 파일을 읽습니다.
kiwi.load_user_dictionary(dict_path = dict_file)

### BoW 생성: 사용자 사전 적용하여 형태소 분석 실행

In [ ]:
# 형태소 분석 결과를 저장할 빈 리스트를 생성합니다.
morphs = []

# 사용자 사전을 적용하여 문서 집합으로 형태소 분석을 실행하고 morphs를 생성합니다.
for corrected in tqdm(newsReply['corrected']):
    tokens = tokenizer(text = corrected)
    morphs.append(tokens)

In [ ]:
# morphs(형태소로 나눈 문서 집합)의 처음 10개 원소를 확인합니다.
for i in range(10):
    print(f'{i}: {newsReply["corrected"].iloc[i]}')
    print(f'{i}: {morphs[i]}', end = '\n\n')

### 바이그램 생성

In [ ]:
# morphs의 첫 번째 원소에서 연속하는 두 단어를 튜플로 묶은 bg(바이그램)를 생성합니다.
bg = bigrams(sequence = morphs[0])

In [ ]:
# bg(바이그램)를 리스트로 변환하여 결과를 확인합니다.
list(bg)

In [ ]:
# bg(바이그램)를 저장할 빈 리스트를 생성합니다.
bgs = []

# morphs에서 연속하는 두 단어를 묶은 튜플을 bgs에 결합합니다.
for i in range(len(morphs)):
    bg = bigrams(sequence = morphs[i])
    bgs += list(bg)

In [ ]:
# bgs(바이그램을 저장한 리스트)를 확인합니다.
bgs

In [ ]:
# bgs를 시리즈로 변환하여 원소별 도수를 계산하고 처음 20개를 확인합니다.
pd.Series(data = bgs).value_counts().head(n = 20)

### 바이그램의 일부 단어를 포함하는 문서 인덱스 확인

In [ ]:
# 사용자 사전에 추가할 단어를 포함하면 True, 그렇지 않으면 False인 cond를 생성합니다.
cond = newsReply['corrected'].str.contains(pat = '골목식당|문어발|빽다방|골목상권|홍콩반점|빽햄')

In [ ]:
# 사용자 사전에 추가할 단어를 포함하는 문서의 인덱스를 확인합니다.
indices = np.where(cond)[0]
print(indices)

In [ ]:
# newsReply corrected에서 indices에 해당하는 문서를 확인합니다.
for i in indices[0:10]:
    print(f'{i}: {newsReply["corrected"].iloc[i]}')
    print(f'{i}: {morphs[i]}', end = '\n\n')

### 사용자 사전용 텍스트 파일에 추가

In [ ]:
# 사용자 사전에 추가할 단어 목록과 단어의 품사 및 점수를 각각 리스트로 생성합니다.
words = ['더본', '골목식당', '빽다방', '홍콩반점', '짜장데이', '빽햄', '남극의 세프', '흑백 요리사', 
         '전문경영인', '장사꾼', '문어발', '골목상권', '소상공인']
tags = ['NNP'] * 8 + ['NNG'] * 5
scores = [1] * 13

In [ ]:
# 사용자 사전을 텍스트 파일로 저장합니다.
# [참고] 단어, 품사, 점수 사이에 구분자(탭)를 설정하고 문장 마지막에 개행문자를 추가합니다.
with open(file = dict_file, mode = 'a') as file:
    for word, tag, score in zip(words, tags, scores):
        file.write(f'{word}\t{tag}\t{score}\n')

In [ ]:
# 사용자 사전에 등록할 텍스트 파일을 읽습니다.
kiwi.load_user_dictionary(dict_path = dict_file)

### BoW 생성: 사용자 사전 적용하여 형태소 분석 실행

In [ ]:
# 형태소 분석 결과를 저장할 빈 리스트를 생성합니다.
morphs = []

# 사용자 사전을 적용하여 문서 집합으로 형태소 분석을 실행하고 morphs를 생성합니다.
for corrected in tqdm(newsReply['corrected']):
    tokens = tokenizer(text = corrected)
    morphs.append(tokens)

In [ ]:
# morphs(형태소로 나눈 문서 집합)의 처음 10개 원소를 확인합니다.
for i in indices[0:10]:
    print(f'{i}: {newsReply["corrected"].iloc[i]}')
    print(f'{i}: {morphs[i]}', end = '\n\n')

### morphs 원소 전처리

In [ ]:
# 형태소(단어)에 있는 공백을 제거하고 morphs에 재할당합니다.
# [참고] 사용자 사전에 추가한 단어(남극의 세프, 흑백 요리사 등)에 공백이 있습니다.
morphs = [[j.replace(' ', '') for j in i] for i in morphs]

In [ ]:
# morphs의 원소(리스트)를 하나의 문자열로 결합하고 corpus에 할당합니다.
corpus = [' '.join(i) for i in morphs]

In [ ]:
# corpus의 처음 10개 원소를 확인합니다.
corpus[0:10]

### 문서-단어 행렬 생성

In [ ]:
# 단어 도수에 역 문서 도수(가중치)를 곱하는 객체를 생성합니다.
tv = TfidfVectorizer(min_df = 0.001)

In [ ]:
# TF-IDF를 성분으로 갖는 문서-단어 행렬을 생성합니다.
dtm = tv.fit_transform(raw_documents = corpus).toarray()

### 문서-단어 행렬 확인

In [ ]:
# dtm을 데이터프레임으로 변환합니다.
dtm = pd.DataFrame(data = dtm, columns = tv.get_feature_names_out())

In [ ]:
# dtm의 처음 10행을 확인합니다.
dtm.head(n = 10)

In [ ]:
# dtm의 행 개수와 열 개수를 확인합니다.
dtm.shape

In [ ]:
# dtm의 단어별 도수 합계를 termFreqs에 할당합니다.
termFreqs = dtm.sum().sort_values(ascending = False)

In [ ]:
# termFreqs의 처음 20행을 확인합니다.
termFreqs.head(n = 20)

### 외부 파일로 저장

In [ ]:
# morphs와 dtm을 pkl 파일로 저장합니다.
pd.to_pickle(obj = [morphs, dtm], filepath_or_buffer = 'Text_Data_Prep.pkl')

In [ ]:
# 현재 작업 경로에 있는 폴더명과 파일명을 확인합니다.
sorted(os.listdir())

## 데이터 시각화

### 시각화 옵션 설정

In [ ]:
# 관련 라이브러리를 호출합니다.
import matplotlib.font_manager as fm
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud

In [ ]:
# 현재 사용 중인 컴퓨터에 설치한 전체 폰트 파일명을 리스트로 반환합니다.
fontList = fm.findSystemFonts(fontext = 'ttf')

In [ ]:
# fontList를 확인합니다.
fontList

In [ ]:
# fontList에서 특정 문자열(폰트명)을 포함하는 파일명을 선택합니다.
fontPath = [font for font in fontList if 'Gowun' in font]

In [ ]:
# fontPath를 확인합니다.
fontPath

In [ ]:
# for 반복문으로 컴퓨터에 설치한 폰트명을 출력합니다.
for font in fontPath:
    print(fm.FontProperties(fname = font).get_name())

In [ ]:
# 한글폰트와 글자 크기를 설정합니다.
plt.rc(group = 'font', family = 'Gowun Batang', size = 10)

In [ ]:
# 그래프 크기와 해상도를 설정합니다.
plt.rc(group = 'figure', figsize = (12, 4), dpi = 150)

### 워드 클라우드 시각화

In [ ]:
# 단어와 TF-IDF 합계를 저장할 빈 딕셔너리를 생성합니다.
freq = dict()

# for 반복문으로 워드 클라우드 시각화용 딕셔너리를 생성합니다.
# [참고] 키는 단어, 값은 TF-IDF 합계입니다.
for word in dtm.columns:
    freq[word] = dtm[word].sum()

In [ ]:
# 워드 클라우드 객체를 생성합니다.
wc = WordCloud(
    font_path = fontPath[1],
    width = 800, 
    height = 800, 
    background_color = 'black', 
    colormap = 'Dark2'
)

In [ ]:
# 워드 클라우드를 시각화합니다.
plt.figure(figsize = (4, 4), dpi = 150)
wcgf = wc.generate_from_frequencies(frequencies = freq)
plt.imshow(X = wcgf)
plt.axis('off')
plt.show()

### 고빈도 단어 시각화

In [ ]:
# 고빈도 상위 20개 단어를 막대 그래프로 시각화합니다.
highFreqs = termFreqs.head(n = 20)
sns.barplot(x = highFreqs.index, y = highFreqs.values, color = 'silver')
plt.title(label = '고빈도 단어 목록(상위 10개)')
plt.xlabel(xlabel = '고빈도 단어')
plt.xlim(-1, 20)
plt.ylim(0, highFreqs.max() * 1.2)
plt.xticks(rotation = 90)
for i, v in enumerate(highFreqs):
    plt.text(x = i, y = v, s = f'{v:.1f}', ha = 'center', va = 'bottom', fontsize = 8);

### 연관 단어 시각화

In [ ]:
# 단어 간 상관계수 행렬을 corMat에 할당합니다.
# [참고] 열(차원) 개수가 많을수록 연산 시간이 길어집니다.
corMat = dtm.corr().round(4)

In [ ]:
# corMat의 일부 행과 열을 확인합니다.
corMat.iloc[0:10, 0:10]

In [ ]:
# 키워드의 연관 단어로 막대 그래프를 그리는 함수를 생성합니다.
def plot_assocs(keyword, n = 20, pal = 'Reds_r'):
    assocs = corMat[keyword].sort_values(ascending = False).head(n = n + 1).iloc[1:]
    max_value = assocs.values.max()
    sns.barplot(x = assocs.index, y = assocs.values, hue = assocs.index, palette = pal)
    plt.title(label = '연관 단어 목록')
    plt.xlabel(xlabel = '고빈도 단어')
    plt.xlim(-1, 20)
    plt.ylim(0, max_value * 1.2)
    plt.xticks(rotation = 90)
    for i, v in enumerate(assocs):
        plt.text(x = i, y = v, s = f'{v:.2f}', ha = 'center', va = 'bottom', fontsize = 8)

In [ ]:
# 키워드의 연관 단어를 막대 그래프로 시각화합니다.
plot_assocs(keyword = '백종원')

In [ ]:
# 키워드의 연관 단어를 막대 그래프로 시각화합니다.
plot_assocs(keyword = '방송')

In [ ]:
# 키워드의 연관 단어를 막대 그래프로 시각화합니다.
plot_assocs(keyword = '사기')

In [ ]:
# 키워드의 연관 단어를 막대 그래프로 시각화합니다.
plot_assocs(keyword = '장사꾼')

## End of Document